In [1]:
# !pip install paddlepaddle-gpu==2.4
!pip install paddlepaddle==2.4
!pip install rocketqa
!/opt/bin/nvidia-smi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 67.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.2/383.2 kB 18.4 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-api-core 2.11.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0.dev0,>=3.19.5, but you have protobuf 3.20.0 which is incompatible.
google-cloud-bigquery 3.10.0 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 3.20.0 which is incompatible.
google-cloud-bigquery-connection 1.12.1 requi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.1/56.1 kB 1.8 MB/s eta 0:00:00
/bin/bash: line 1: /opt/bin/nvidia-smi: No such file or directory


In [2]:
import torch
print(torch.__version__)
print(torch.version.cuda)
print(torch.backends.cudnn.version())

2.0.1+cu118
11.8
8700


In [3]:
# # #Uninstall the current CUDA version
# !apt-get --purge remove cuda nvidia* libnvidia-*
# !dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
# !apt-get remove cuda-*
# !apt autoremove
# !apt-get update

# #Download CUDA 10.0
# !wget  --no-clobber https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/cuda-repo-ubuntu1804_10.2.89-1_amd64.deb
# #install CUDA kit dpkg
# !yes | dpkg -i cuda-repo-ubuntu1804_10.2.89-1_amd64.deb
# !sudo apt-key adv --fetch-keys https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/7fa2af80.pub
# !apt-get update
# !apt-get install cuda-10-2

In [4]:
!nvcc --version
!cat /usr/local/cuda/version.txt
!cat /usr/include/x86_64-linux-gnu/cudnn_v*.h | grep CUDNN_MAJOR -A 2

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0
cat: /usr/local/cuda/version.txt: No such file or directory
#define CUDNN_MAJOR 8
#define CUDNN_MINOR 9
#define CUDNN_PATCHLEVEL 0
--
#define CUDNN_VERSION (CUDNN_MAJOR * 1000 + CUDNN_MINOR * 100 + CUDNN_PATCHLEVEL)

/* cannot use constexpr here since this is a C-only file */


In [5]:
from google.colab import drive
drive.mount('/mntDrive')

import rocketqa
import json
import os
import numpy as np

Mounted at /mntDrive


In [6]:
data_p =  '/mntDrive/MyDrive/Colab Notebooks/'
passages_dir = data_p + "data/groundtruth/passages_proc_table_0708"
groundtruth_fp = data_p +  "data/groundtruth/qa_groundtruth.json"

para_embs_dir = data_p + "data/groundtruth/rocketqa/para_proc_table_embs"
# query_embs_groundtruth = data_p + "data/groundtruth/rocketqa/query_embs_groundtruth"


In [7]:
def get_passage_embds(model_name, date, passages_dir_prefix, para_embs_dir_prefix):
    passage_dir = passages_dir_prefix.format(date)
    para_embs_dir = para_embs_dir_prefix.format(model_name, date)

    if not os.path.exists(para_embs_dir):
        os.makedirs(para_embs_dir)

    # device = "cuda:0" if torch.cuda.is_available() else "cpu"
    device = "cpu"
    if device == "cuda:0":
        use_cuda = True
    else:
        use_cuda = False

    # init dual encoder
    print("init")
    dual_encoder = rocketqa.load_model(model="v1_marco_de", use_cuda=use_cuda, device_id=0, batch_size=16)

    for doc in os.listdir(passage_dir):
        print(doc)
        doc_fp = os.path.join(passage_dir, doc)
        save_fp = os.path.join(para_embs_dir, doc.split('.')[0])
        # if os.path.exists(save_fp):
        #   continue
        with open(doc_fp, "r") as f:
            passages = json.load(f)

        ptxt_list = []
        pctxt_list = []

        for pid in range(len(passages["passages_text"])):
            ptxt_list.append(passages["passages_text"][pid]["ptxt"])
            if "passages_composite_text" in passages:
                pctxt_list.append(passages["passages_composite_text"][pid]["pctxt"])

        ptxt_embs = list(dual_encoder.encode_para(para=ptxt_list))
        pctxt_embs = list(dual_encoder.encode_para(para=pctxt_list))
        embs = {"ptxt_embs": ptxt_embs, "pctxt_embs": pctxt_embs}
        # np.savez(save_fp, **embs)
        with open(os.path.join(para_embs_dir, doc), "w") as f:
            json.dump({"ptxt_embs": ptxt_embs, "pctxt_embs": pctxt_embs}, f)

        # encode para

        # ptxt_embs_list = list(dual_encoder.encode_para(para=ptxt_list))
        # pctxt_embs_list = list(dual_encoder.encode_para(para=pctxt_list))
        # embs = {"passages_text":[], "passages_composite_text": []}
        # for pid in range(len(ptxt_embs_list)):
        #     embs["passages_text"].append({"pid": pid, "pemb": ptxt_embs_list[pid]})
        #     embs["passages_composite_text"].append({"pid": pid, "pcemb": pctxt_embs_list[pid]})
        # save_fp = os.path.join(para_embs_dir, doc.split('.')[0])
        # np.savez(save_fp, **embs)
        # print(save_fp)

In [8]:
# import rocketqa
# device = "cuda:0" if torch.cuda.is_available() else "cpu"

# model_name = 'rocketqa-v1_marco_de'
# model = rocketqa.load_model(model="v1_marco_de", use_cuda=False, device_id=0, batch_size=16)
# total_params = sum(p.numel() for p in model.parameters())
# print(model_name, ": ", total_params)

In [9]:
data_p =  '/mntDrive/MyDrive/Colab Notebooks/'

date = "0708"

passages_dir_prefix = data_p + "data/groundtruth/passages_{0}"
para_embs_dir_prefix = data_p + "data/groundtruth/{0}/para_embs_{1}"

# passages_dir_prefix = data_p + "data/groundtruth/passages_proc_table_{0}"
# para_embs_dir_prefix = data_p + "data/groundtruth/{0}/para_proc_table_embs_{1}"

# passages_dir_prefix = data_p + "data/groundtruth/passages_chunk_{0}"
# para_embs_dir_prefix = data_p + "data/groundtruth/{0}/para_embs_chunk_{1}"

# passages_dir_prefix = data_p + "data/groundtruth/passages_table2text_tid_{0}"
# para_embs_dir_prefix = data_p + "data/groundtruth/{0}/para_embs_table2text_{1}"

groundtruth_fp = data_p +  "data/groundtruth/qa_groundtruth.json"
query_emb_prefix =  data_p + "data/groundtruth/{0}/query_emb_{1}.json"

model_name = 'rocketqa'



get_passage_embds(model_name, date, passages_dir_prefix, para_embs_dir_prefix)

init


RocketQA model [v1_marco_de]
Download RocketQA model [v1_marco_de]
100%|███████████████████████████████████████| 769M/769M [00:36<00:00, 22.1MiB/s]
Load model done


280cb2e157d1e8b4586809fb14861f04.json


TypeError: ignored